In [1]:
import matplotlib.pylab as plt
# import networkx as nx
import numpy as np
import pandas as pd
import time
import sys
import os
import textwrap

In [2]:
# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
import tmv_app.models as tm
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Case, When, Value as V
from django.db import models

In [10]:
% matplotlib inline

# topic proportions according to party
# copied from parliament/views -> adapted for utterance
run_id = 2111
run = RunStats.objects.get(run_id=run_id)
method = run.method
K = run.K

colors = []
party_scores = []
party_names = []
norm_scores = []
topic_ids = []
topic_list = []
total_scores = []

for topic in tm.Topic.objects.filter(run_id=run_id).order_by('score'):
    # print(topic)
    s = tm.RunStats.objects.get(run_id=run_id).psearch
    party_totals = pm.Utterance.objects.filter(
        search_matches=s,
        doctopic__topic__run_id=run_id,
        speaker__party__name__isnull=False,
        speaker__party__colour__isnull=False
    ).order_by().values('speaker__party__name').annotate(
        topic_score=Sum(
            Case(
                When(doctopic__topic=topic, then=F('doctopic__score')),
                default=0,
                output_field=models.FloatField()
            )
        ),
        total_score=Sum('doctopic__score'),
    ).annotate(
        topic_proportion=F('topic_score') / F('total_score')
    ).values(
        'topic_proportion',
        'speaker__party__id',
        'speaker__party__name',
        'speaker__party__colour'
    )

    colors.append([d['speaker__party__colour'] for d in party_totals])
    party_names.append([d['speaker__party__name'] for d in party_totals])
    sc = [d['topic_proportion'] for d in party_totals]
    party_scores.append(sc)
    norm_scores.append(np.array(sc) / sum(sc))
    topic_ids.append(topic.id)
    topic_list.append(topic.top_words)
    total_scores.append(topic.score)

NameError: name 'RunStats' is not defined

In [4]:
print(party_scores)
#print(party_names)
print(topic_ids)
np.array(party_scores).sum(0)

[[0.000471976081786036, 0.00116065922602316, 0.00180974294180827, 0.00116698067188348, 0.000984130951075586, 0.00133887669336582], [0.122351927100081, 0.00145350926876908, 0.0028283809708849, 0.00555672921661024, 0.00465525159030027, 0.00126469358916083], [0.000452614751022755, 0.00269305262494404, 0.00155356789075124, 0.00190661364121743, 0.00320871942955242, 0.00361558145008075], [0.00106938996990557, 0.00394662417432415, 0.00251365774564045, 0.0021710710754938, 0.00182513003718804, 0.00266251226131384], [0.00187654057817592, 0.00337675348006166, 0.00245271638639257, 0.0036325740107917, 0.00227819928470429, 0.00454590009675649], [0.00115294445749302, 0.00439928707415244, 0.00428255251150871, 0.000459693739734479, 0.000708217557587938, 0.00521303879734984], [0.0140922151258981, 0.00449395866144915, 0.00150453562957633, 0.0134381305414183, 0.00808292576098897, 0.00637210675733095], [0.00201776850404599, 0.00487371795806706, 0.00485592201850575, 0.0111937594178159, 0.0236681808851584, 0

array([1., 1., 1., 1., 1., 1.])

In [5]:
# similarly for regions of speakers
# -> this implementation is not working but should be faster, when debugged

'''
region_scores = []
topic_ids2 = []

coal_regions = cmodels.Region.objects.filter(name__in=["Brandenburg", "North Rhine-Westphalia", "Saxony"])
non_coal_regions = cmodels.Region.objects.filter(name__in=["Baden-Württemberg", "Bavaria", "Berlin", "Bremen", "Hamburg", "Hesse",
                    "Mecklenburg-Vorpommern", "Lower Saxony", "Rheinland-Pfalz",
                    "Saarland", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"])

topics = tm.Topic.objects.filter(run_id=run_id).order_by('score')[0:2]

arrays = [range(len(topics)),
          ['direct', 'list', 'sum']]

index = pd.MultiIndex.from_product(arrays, names=['topic #', 'seat_type'])
df_regions = pd.DataFrame(index=index, columns=['coal', 'ncoal', 'top_words'])

index_to_keyword = {0: 'coal', 1: 'ncoal'}
for i, topic in enumerate(topics):
    #print(topic.top_words)
    
    region_scores_collect = []
    
    for j, regions in enumerate([coal_regions, non_coal_regions]):
        #print(list(regions))

        # differentiate between different seat types (list or direct mandate):
        # list: speaker -> seat -> partylist -> region
        ut_list = pm.Utterance.objects.filter(search_matches=s,
                            doctopic__topic__run_id=run_id,
                            speaker__seat__seat_type=2).distinct()
        ut_list = ut_list.filter(speaker__seat__list__region__in=regions)

        # direct: speaker -> seat -> consituency -> region
        ut_direct = pm.Utterance.objects.filter(search_matches=s,
                            doctopic__topic__run_id=run_id,
                            speaker__seat__seat_type=1).distinct()
        ut_direct = ut_direct.filter(speaker__seat__constituency__region__in=regions)
        ut = ut_list.union(ut_direct)

        region_score = ut.order_by().values('speaker__seat__seat_type').annotate(
                topic_score=models.Sum(
                    models.Case(
                        models.When(doctopic__topic=topic, then=models.F('doctopic__score')),
                        default=0,
                        output_field=models.FloatField()
                    )
                ),
                total_score=models.Sum('doctopic__score'),
            ).annotate(
                topic_proportion=models.F('topic_score') / models.F('total_score')
            ).values('topic_proportion')

        print(region_score)
        region_scores_collect.append(region_score)
    
    region_scores.append(region_scores_collect)
    topic_ids2.append(topic.id)
'''

'\nregion_scores = []\ntopic_ids2 = []\n\ncoal_regions = cmodels.Region.objects.filter(name__in=["Brandenburg", "North Rhine-Westphalia", "Saxony"])\nnon_coal_regions = cmodels.Region.objects.filter(name__in=["Baden-Württemberg", "Bavaria", "Berlin", "Bremen", "Hamburg", "Hesse",\n                    "Mecklenburg-Vorpommern", "Lower Saxony", "Rheinland-Pfalz",\n                    "Saarland", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"])\n\ntopics = tm.Topic.objects.filter(run_id=run_id).order_by(\'score\')[0:2]\n\narrays = [range(len(topics)),\n          [\'direct\', \'list\', \'sum\']]\n\nindex = pd.MultiIndex.from_product(arrays, names=[\'topic #\', \'seat_type\'])\ndf_regions = pd.DataFrame(index=index, columns=[\'coal\', \'ncoal\', \'top_words\'])\n\nindex_to_keyword = {0: \'coal\', 1: \'ncoal\'}\nfor i, topic in enumerate(topics):\n    #print(topic.top_words)\n    \n    region_scores_collect = []\n    \n    for j, regions in enumerate([coal_regions, non_coal_regions]):\n  

In [6]:
# this implementation is working but really slow
# because it makes queries to the database for every topic and region separately

recalc = True

if recalc:

    coal_regions = cmodels.Region.objects.filter(name__in=["Brandenburg", "North Rhine-Westphalia", "Saxony"])
    non_coal_regions = cmodels.Region.objects.filter(name__in=["Baden-Württemberg", "Bavaria", "Berlin", "Bremen", "Hamburg", "Hesse",
                        "Mecklenburg-Vorpommern", "Lower Saxony", "Rheinland-Pfalz",
                        "Saarland", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"])

    topics = tm.Topic.objects.filter(run_id=run_id).order_by('score')
    seat_types = ['direct', 'list', 'sum']
    arrays = [range(len(topics)), seat_types]

    index = pd.MultiIndex.from_product(arrays, names=['topic #', 'seat_type'])
    df_regions = pd.DataFrame(index=index, columns=['coal', 'ncoal', 'topic_id', 'top_words'])

    index_to_keyword = {0: 'coal', 1: 'ncoal'}


    for i, topic in enumerate(topics):
        print(topic.top_words)

        for j, regions in enumerate([coal_regions, non_coal_regions]):
            print(list(regions))

            region_score_list = pm.Utterance.objects.filter(
                    search_matches=s,
                    doctopic__topic__run_id=run_id,
                    speaker__seat__seat_type=2,
                    speaker__seat__list__region__in=regions
                ).distinct().order_by().values('speaker__seat__seat_type').annotate(
                    topic_score=models.Sum(
                        models.Case(
                            models.When(doctopic__topic=topic, then=models.F('doctopic__score')),
                            default=0,
                            output_field=models.FloatField()
                        )
                    ),
                    total_score=models.Sum('doctopic__score'),
                ).annotate(
                    topic_proportion=models.F('topic_score') / models.F('total_score')
                )#.values('topic_proportion')

            region_score_direct = pm.Utterance.objects.filter(
                    search_matches=s,
                    doctopic__topic__run_id=run_id,
                    speaker__seat__seat_type=1,
                    speaker__seat__constituency__region__in=regions
                ).distinct().order_by().values('speaker__seat__seat_type').annotate(
                    topic_score=models.Sum(
                        models.Case(
                            models.When(doctopic__topic=topic, then=models.F('doctopic__score')),
                            default=0,
                            output_field=models.FloatField()
                        )
                    ),
                    total_score=models.Sum('doctopic__score'),
                ).annotate(
                    topic_proportion=models.F('topic_score') / models.F('total_score')
                )#.values('topic_proportion')

            for st in seat_types:
                df_regions['top_words'][(i, st)] = topic.top_words
                df_regions['topic_id'][(i, st)] = topic.id

            df_regions[index_to_keyword[j]][(i, 'direct')] = region_score_direct[0]['topic_proportion']
            df_regions[index_to_keyword[j]][(i, 'list')] = region_score_list[0]['topic_proportion']
            total_score = region_score_list[0]['topic_proportion'] + region_score_direct[0]['topic_proportion'] / 2.
            df_regions[index_to_keyword[j]][(i, 'sum')] = total_score

            print("score list: {}".format(region_score_list[0]['topic_proportion']))
            print("score direct: {}".format(region_score_direct[0]['topic_proportion']))
            print("sum: {}".format(total_score))


None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.00183652485963949
score direct: 0.00200482931739551
sum: 0.002838939518337245
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpommern>, <Region: Rheinland-Pfalz>, <Region: Saarland>, <Region: Saxony-Anhalt>, <Region: Schleswig-Holstein>, <Region: Thuringia>]
score list: 0.00108002384583164
score direct: 0.000880596604360459
sum: 0.001520322148011886
None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.00220475868636025
score direct: 0.000875023719762755
sum: 0.002642270546241628
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpommern>, <Region: Rheinland-Pfalz>, <Region: Saarland>, <Region: Saxony-Anhalt>, <Region

score list: 0.00879671778287698
score direct: 0.00353238078969734
sum: 0.01056290817772555
None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.00365660091364535
score direct: 0.00631101582093176
sum: 0.00681210882411123
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpommern>, <Region: Rheinland-Pfalz>, <Region: Saarland>, <Region: Saxony-Anhalt>, <Region: Schleswig-Holstein>, <Region: Thuringia>]
score list: 0.00899124293361964
score direct: 0.0159067048172747
sum: 0.0169445953422565
None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.00354397258480904
score direct: 0.00706246127559878
sum: 0.00707520322260843
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpomme

score list: 0.0149826838825289
score direct: 0.0112775263096113
sum: 0.0206214470373339
None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.0193651433712992
score direct: 0.0127279314120649
sum: 0.02572910907733165
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpommern>, <Region: Rheinland-Pfalz>, <Region: Saarland>, <Region: Saxony-Anhalt>, <Region: Schleswig-Holstein>, <Region: Thuringia>]
score list: 0.0330474308094972
score direct: 0.0279799655783247
sum: 0.04703741359866035
None
[<Region: Brandenburg>, <Region: North Rhine-Westphalia>, <Region: Saxony>]
score list: 0.0239067431737414
score direct: 0.0155371651142909
sum: 0.03167532573088685
[<Region: Baden-Württemberg>, <Region: Bavaria>, <Region: Berlin>, <Region: Bremen>, <Region: Hamburg>, <Region: Hesse>, <Region: Lower Saxony>, <Region: Mecklenburg-Vorpommern>, <R

score list: 0.223314061343687
score direct: 0.222321879892449
sum: 0.334475001289884


In [ ]:
if recalc:
    regions_data_filename = "regions_scores_{}_K{}_run_id{}.csv".format(method, K, run_id)
    df_regions.to_csv(regions_data_filename)

    if os.path.isfile(regions_data_filename):
        df_regions_test = pd.read_csv(regions_data_filename, index_col=[0,1], skipinitialspace=True)
    df_regions_test

In [ ]:
df_regions_test

In [33]:
if not recalc:
    regions_data_filename = "regions_scores_{}_K{}_run_id{}.csv".format(method, K, run_id)
    df_regions = pd.read_csv(regions_data_filename, index_col=[0,1], skipinitialspace=True)

In [ ]:
idx = pd.IndexSlice
region_data = df_regions.loc[idx[:,'sum'],['coal','ncoal']].values
normalized_region_data = np.array([i / sum(i) for i in region_data])

In [ ]:
# plot list of topics and visualize scores for parties and regions

k = len(topic_ids)

# indices of topics to plot
#index_list = [0,3,7,8,9,10,12,13, 14, 15, 16, 18, 22, 24]
index_list = range(k)
plotted_topics = len(index_list)
y_pos = np.arange(plotted_topics)
fig, axs = plt.subplots(1, 8, figsize=(15,plotted_topics/2), sharey=True)

# topic description
ax = axs[0]
# numbering from 1 to ...
description_list = np.arange(k, 0, -1)
# indices
#description_list = np.arange(0, k)
# topic id in database
#description_list = np.array(topic_id)


box_props = dict(boxstyle='round', fc="w", ec="k")
indexed_topic_list = np.array(topic_list)[index_list]
for i, text in enumerate(description_list[index_list]):
    ax.text(-0.1, y_pos[i], text,
            horizontalalignment='right',
            verticalalignment='center',
           fontsize=16)
    topic_text = textwrap.fill(" ".join(indexed_topic_list[i]), width=55)
    if len(topic_text.split("\n")) > 2:
        topic_text = "\n".join(topic_text.split("\n")[0:2]) + " ..."
    text_element = ax.text(0, y_pos[i], topic_text, fontsize=10,
        verticalalignment='center', horizontalalignment='left')

ax.set_axis_off()
ax.set_title("      id      most frequent topic terms", loc='center')

ax = axs[1]
ax.set_axis_off()
ax = axs[2]
ax.set_axis_off()

# total scores
ax = axs[3]
patch_handles = []
ax.barh(y_pos, np.array(total_scores)[index_list], align='center')
ax.set_title("topic score", loc='left')

# party proportions
ax = axs[4]
data = np.array(party_scores)[index_list].T
plot_colors = np.array(colors)[index_list].T
patch_handles = []
left = np.zeros(plotted_topics) # left alignment of data starts at zero
for i, d in enumerate(data):
    patch_handles.append(ax.barh(y_pos, d, 
      color=plot_colors[i], align='center', 
      left=left))
    # accumulate the left-hand offsets
    left += d
ax.set_title("party proportions")


# normalized party scores
ax = axs[5]
data = np.array(norm_scores)[index_list].T
plot_colors = np.array(colors)[index_list].T
patch_handles = []
left = np.zeros(plotted_topics) # left alignment of data starts at zero
for i, d in enumerate(data):
    patch_handles.append(ax.barh(y_pos, d, 
      color=plot_colors[i], align='center',
      left=left))
    # accumulate the left-hand offsets
    left += d
ax.set_title("pp normalized")

# region scores
ax = axs[6]
data = np.array(region_data)[index_list].T
plot_colors = np.tile(['limegreen', 'saddlebrown'], (plotted_topics,1)).T
patch_handles = []
left = np.zeros(plotted_topics) # left alignment of data starts at zero
for i, d in enumerate(data):
    patch_handles.append(ax.barh(y_pos, d, 
      color=plot_colors[i], align='center',
      left=left))
    # accumulate the left-hand offsets
    left = left + d
ax.set_title("region proportions")

# normalized region scores
ax = axs[7]
data = np.array(normalized_region_data)[index_list].T
plot_colors = np.tile(['limegreen', 'saddlebrown'], (plotted_topics,1)).T
patch_handles = []
left = np.zeros(plotted_topics) # left alignment of data starts at zero
for i, d in enumerate(data):
    patch_handles.append(ax.barh(y_pos, d, 
      color=plot_colors[i], align='center',
      left=left))
    # accumulate the left-hand offsets
    left = left + d
ax.plot([0.5, 0.5], [-0.5, plotted_topics-0.5], 'k--')
ax.set_title("rp normalized")

for ax in axs[3:8]:
    ax.yaxis.set_visible(False) # Hide only x axis
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)

ax.set_ylim(-0.5, plotted_topics)

fig.savefig("topic_list_{}_{}_K{}_run_id{}.png".format(search_word, method, K, run_id), 
            bbox_inches='tight', dpi=150)

In [ ]:
region_data.sum(0)

In [ ]:
region_data

In [ ]:
plot_colors

In [ ]:
# todo: make more efficient by requesting all data at once
party_totals = pm.Utterance.objects.filter(
    search_matches=s,
    doctopic__topic__run_id=stat,
    speaker__party__name__isnull=False,
    speaker__party__colour__isnull=False
).order_by().values('speaker__party__name').annotate(
    topic_score=Sum(
        Case(
            When(doctopic__topic=topic, then=F('doctopic__score')),
            default=0,
            output_field=models.FloatField()
        )
    ),
    total_score=Sum('doctopic__score'),
).annotate(
    topic_proportion=F('topic_score') / F('total_score')
).values(
    'topic_proportion',
    'speaker__party__id',
    'speaker__party__name',
    'speaker__party__colour'
)